In [ ]:
import sys

from format_markdown import markdown

sys.path.insert(0, "..")

In [ ]:
from versions import (
    JUPYTER_LSP_VERSION,
    JUPYTERLAB_LSP_VERSION,
    JUPYTERLAB_NEXT_MAJOR_VERSION,
    JUPYTERLAB_VERSION,
    REQUIRED_JUPYTER_SERVER,
    REQUIRED_JUPYTERLAB,
    REQUIRED_PYTHON,
)

## Installation

### Please Read This First

Delivering LSP features to your JupyterLab **requires** three pieces:

#### 1. `jupyter-lsp`

In [ ]:
%%markdown
- runs in your `jupyter_server` application on your server to handle requests from
  the browser to _language servers_
- to run, you need:
  - `python {REQUIRED_PYTHON}`
  - `jupyter_server {REQUIRED_JUPYTER_SERVER}`

#### 2. `jupyterlab-lsp`

In [ ]:
%%markdown
- runs in your browser, as an extension to JupyterLab
- to install it, you need:
  - `jupyterlab {REQUIRED_JUPYTERLAB}`

#### 3. Language Servers

- run on your server
  - probably in another language runtime than python
- some can be automatically [detected](./Language%20Servers.html) if installed
- others also need to be [configured](./Configuring.html#language_servers)

### Clean installation

The approaches demonstrated below will ensure that correct versions of Python,
JupyterLab and extensions are installed, and are generally recommended for
novice users, or users who are not familiar with the Python ecosystem. However,
if you feel proficient with using Python package managers such as `pip` and
`conda` you may prefer to follow the [custom installation](#custom-installation)
guide instead, which allows to install the packages in an existing environment.

#### conda (minimal python)

When installing from conda-forge, the `jupyter-lsp-python` bundle includes both
the server extension (`jupyter-lsp`) and `pyls` (a third-party server also known
as `python-language-server`). You can swap `jupyter-lsp-python` with another
pre-made bundle, `jupyter-lsp-r` to get get the same server extension but with
`r-languageserver` instead. Alternatively, you can install a language server of
your choice manually (see further steps).

In [ ]:
%%markdown
```bash
conda create -c conda-forge -n lsp 'python {REQUIRED_PYTHON}' 'jupyterlab={JUPYTERLAB_VERSION}' 'jupyterlab-lsp={JUPYTERLAB_LSP_VERSION}' 'jupyter-lsp-python={JUPYTER_LSP_VERSION}'
conda activate lsp
```

Then run

```bash
jupyter lab
```

Your browser should open to your local server.

#### docker (data science)

This approach is based roughly on the
[Jupyter docker-stacks documentation](https://github.com/jupyter/docker-stacks/tree/master/examples/docker-compose/notebook),
which should be consulted for more about connecting volumes, passwords, and
other advanced features:

> Note: docker instructions were **not** updated for JupyterLab 3.0 and
> extension 3.0. Please consider submitting a PR to fix it.

##### `Dockerfile`

In [ ]:
%%markdown

```dockerfile
# This already contains the python, r, julia, latex, and nodejs runtimes
FROM jupyter/datascience-notebook@sha256:73a577b006b496e1a1c02f5be432f4aab969c456881c4789e0df77c89a0a60c2

RUN conda install --quiet --yes --freeze-installed -c conda-forge \
    'python-language-server' \
    'jupyterlab={JUPYTERLAB_VERSION}' \
    'r-languageserver' \
    'texlab' \
    'chktex' \
    'jupyter-lsp={JUPYTER_LSP_VERSION}' \
  && jupyter labextension install --no-build \
    '@jupyter-lsp/jupyterlab-lsp@{JUPYTERLAB_LSP_VERSION}' \
  && jupyter lab build --dev-build=False --minimize=True \
  && conda clean --all -f -y \
  && rm -rf \
    $CONDA_DIR/share/jupyter/lab/staging \
    /home/$NB_USER/.cache/yarn \
  && fix-permissions $CONDA_DIR \
  && fix-permissions /home/$NB_USER
```

##### `docker-compose.yml`

```yaml
version: '2'

services:
  lsp-lab:
    build: .
    ports:
      - '18888:8888'
```

##### Build and Start

```bash
docker-compose up
```

You should now be able to access `http://localhost:18888/lab`, using the `token`
provided in the log.

### Custom installation

#### Get a working JupyterLab environment

Refer to the official
[JupyterLab Installation Documentation](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)
for your installation approach.

| pip            | conda            | pipenv | poetry | `*` |
| -------------- | ---------------- | ------ | ------ | --- |
| [lab][lab-pip] | [lab][lab-conda] | `*`    | `*`    | `*` |

> `*` PRs welcome!

Verify your lab works:

```bash
jupyter lab --version
jupyter lab
```

[lab-conda]:
  https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#conda
[lab-pip]:
  https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#pip

#### Install Jupyter[Lab] LSP

##### conda

In [ ]:
%%markdown
```bash
conda install jupyterlab-lsp={JUPYTERLAB_LSP_VERSION}
```

##### pip

In [ ]:
%%markdown
```bash
pip install jupyterlab-lsp=={JUPYTERLAB_LSP_VERSION}
```

### Next step: Language Servers

Now that you have `jupyterlab-lsp`, `jupyter-lsp` and all of their dependencies,
you'll need some language servers. See:

- [Language Servers](./Language%20Servers.ipynb) that will be found
  automatically once installed
- [configuring](./Configuring.ipynb) `jupyter-lsp` for more control over which
  servers to load